Instructions: click restart and run all above. Figures will show once the entire notebook has finished running

In [12]:
%pip install pde_rk
import numpy as np
from pde_rk import pde_rk
import pandas as pd
import plotly.express as px

Keyring is skipped due to an exception: 'EntryPoints' object has no attribute 'get'
Note: you may need to restart the kernel to use updated packages.


# Defining and solving PDE models

This notebook will demonstrate how we can define and solve a PDE model using functions in this package.

## pde_rk function

This package contains the pde_rk function, a versatile PDE solver that can be used to solve any system of PDEs, using an adaptive Runge-Kutta method.

In [2]:
help(pde_rk)

Help on function pde_rk in module pde_rk.pde:

pde_rk(dxdt: <built-in function callable>, X0: list, Tmax: float, deltat: float, t_eval: numpy.ndarray, killfunc: Union[<built-in function callable>, NoneType] = None, stabilitycheck: bool = False, maxstep: Union[float, NoneType] = None, rk: bool = True) -> Tuple[list, float, list, numpy.ndarray]
    Function for solving system of PDEs using adaptive Runge-Kutta method
    Adapted from Hubatsch et al., 2019 (see https://github.com/lhcgeneva/PARmodelling)
    
    Args:
        dxdt: a function that takes list of 1D arrays (one for each species) corresponding to concentrations over space, and returns a list of gradient arrays
        X0: a list specifying the initial state of the system. Will be used as the input to dxdt on the first time step
        Tmax: timepoint at which to terminate simulation
        deltat: initial timestep (this will be adapted throughout the simulation)
        t_eval: a list of timepoints for which to save the st

We can use this function to perform custom simulations by building an appropriate PDE model. In this notebook, we will demonstrate this using the Goehring et al., 2011 PAR model as an example.

## Building a PDE model

The primary input of pde_rk is a PDE model function, which defines the reactions in the model. This must take a single argument, representing the current state of the system (X), and return a single output representing the gradient of chage of the system with respect to time (dXdt). X must take the form of a list, where each entry in the list is a 1D array representing the spatial distribution of one species. This can be any length, depending on how many species there are in the model. Here, as we have two species (A and P), X will be a list of length 2. dXdt will take the same form.

As this particular model contains many parameters, it is useful to build it in class form, specifying parameter values in the init function. This is demonstrated below for the Goehring model, where the function dxdt is built to describe all of the different reactions in the model. We can simulate diffusion (reflective boundaries) with the diffusion function, which takes a single concentration array and the spatial step size as inputs, and calculates diffusion for a single timestep.

In [3]:
def diffusion(concs, dx):
    concs_ = np.r_[concs[0], concs, concs[-1]]  # Dirichlet boundary conditions
    d = concs_[:-2] - 2 * concs_[1:-1] + concs_[2:]
    return d / (dx ** 2)

class PAR:
    def __init__(self, Da=0.1, Dp=0.1, konA=1, koffA=0.3, konP=1, koffP=0.3, kPA=2, kAP=2,
                  alpha=2, beta=2, xsteps=100, psi=0.3, L=50, pA=1, pP=1):
     
        # Dosages
        self.pA = pA
        self.pP = pP

        # Diffusion
        self.Da = Da  # input is um2 s-1
        self.Dp = Dp  # um2 s-1

        # Membrane exchange
        self.konA = konA  # um s-1
        self.koffA = koffA  # s-1
        self.konP = konP  # um s-1
        self.koffP = koffP  # s-1

        # Antagonism
        self.kPA = kPA  # um4 s-1
        self.kAP = kAP  # um2 s-1
        self.alpha = alpha
        self.beta = beta

        # Spatial
        self.L = L
        self.xsteps = int(xsteps)
        self.deltax = self.L / xsteps  # um
        self.psi = psi  # um-1

    def dxdt(self, X):
        """
        Function describing time evolution of the model
        X = [A, P], where A and P are arrays of length xsteps, representing cortical aPAR and pPAR concentrations
        
        """
        
        A = X[0]
        P = X[1]
        ac = self.pA - self.psi * np.mean(A)
        pc = self.pP - self.psi * np.mean(P)
        dA = ((self.konA * ac) - (self.koffA * A) - (self.kAP * (P ** self.alpha) * A) + (
                self.Da * diffusion(A, self.deltax)))
        dP = ((self.konP * pc) - (self.koffP * P) - (self.kPA * (A ** self.beta) * P) + (
                self.Dp * diffusion(P, self.deltax)))
        return [dA, dP]
    
model = PAR()

## Specify initial conditions

Simulations must be started from an initial state. There are many possible ways to do this. Here, we will start the system completely polarised with an arbitrary membrane concentration, although there are more systematic ways that we could do this. By starting the simulation from this state, we will test the ability of the model to maintain polarity once given an initial pattern.

In [4]:
def initial_conditions(model):
    A0 = 3 * np.r_[np.ones([model.xsteps // 2]), np.zeros([model.xsteps // 2])]
    P0 = 3 * np.r_[np.zeros([model.xsteps // 2]), np.ones([model.xsteps // 2])] 
    return [A0, P0]

X0 = initial_conditions(model)

In [5]:
df = pd.DataFrame({'Position': np.arange(len(X0[0])), 'aPAR': X0[0], 'pPAR': X0[1]})
fig = px.line(data_frame=df, x="Position", y=['aPAR', 'pPAR'], width=500, height=300, labels={"value": "Membrane concentration"})
fig.update_layout(legend_title="")
fig.show()

## Run PDE simulation

Now that we've set up a model and specified the initial conditions, we can perform a simulation using pdeRK. We also need to specify Tmax, deltat and t_eval. We will run this model for 1000 seconds (~17 mins).

In [6]:
soln, time, solns, times = pde_rk(dxdt=model.dxdt, X0=X0, Tmax=100, deltat=0.01, t_eval=np.arange(0, 101, 1))

In [7]:
df2 = pd.DataFrame({'Position': np.arange(len(soln[0])), 'aPAR': soln[0], 'pPAR': soln[1]})
fig = px.line(data_frame=df2, x="Position", y=['aPAR', 'pPAR'], width=500, height=300, labels={"value": "Membrane concentration"})
fig.update_layout(legend_title="")
fig.show()

#### Plot time evolution

In [8]:
df3 = pd.DataFrame({'Time': times.astype(int), 'Position': [np.arange(len(soln[0]))] * len(times), 
                    'aPAR': list(solns[0]), 'pPAR': list(solns[1])}).explode(column=['Position', 'aPAR', 'pPAR'])
fig = px.line(data_frame=df3, x='Position', y=['aPAR', 'pPAR'], width=500, height=400, labels={"value": "Membrane concentration"}, 
              animation_frame='Time')
fig["layout"].pop("updatemenus")
fig.update_layout(legend_title="")
fig.show()

We can see that, with the current parameters set, the model is succesfully able to maintain polarity if started from a polarised state.

## Running the PDE with a kill function

In the above example, we speficied simulation time with the Tmax parameter, and ran the model until this time was reached. In some cases it may not be necessary to run a model for the whole time period, and we can save time by terminating the simulation early. 

One way to do this is by providing a kill function to pdeRK, with the killfunc argument.  This function will be evaluated at every iteration, taking the same input as dxdt, and test specific criteria for terminating the simulation. In this example, we will set up a function that detects whether or not the system is polarised, returning False is the system is polarised, and True when polarity is lost (defined as the presence or absence of a cross point in the aPAR and pPAR concentration profiles). This will cause the simulation to terminate early if polarity is lost before Tmax. This way we can quickly test whether a particular model can maintain polarity, without having to run simulations for the full time period.

In [9]:
def killfunc(X):
    """
    Returns True if there is no cross point in the aPAR and pPAR concentration profiles
    
    """
    if sum(X[0] > X[1]) == len(X[0]) or sum(X[0] > X[1]) == 0:
        return True
    return False

This is demonstrated below, using a model with a lower kPA that cannot maintain polarity:

In [10]:
model2 = PAR(pA=0.5)

soln2, time2, solns2, times2 = pde_rk(dxdt=model2.dxdt, X0=initial_conditions(model2), Tmax=1000, deltat=0.01, 
                                 t_eval=np.arange(0, 101, 1), killfunc=killfunc)
print(time2)

13.182149765147567


In [11]:
df4 = pd.DataFrame({'Time': np.around(times2, 1), 'Position': [np.arange(len(soln2[0]))] * len(times2), 
                    'aPAR': list(solns2[0]), 'pPAR': list(solns2[1])}).explode(column=['Position', 'aPAR', 'pPAR'])
fig = px.line(data_frame=df4, x='Position', y=['aPAR', 'pPAR'], width=500, height=400, labels={"value": "Membrane concentration"}, 
              animation_frame='Time')
fig["layout"].pop("updatemenus")
fig.update_layout(legend_title="")
fig.show()